In [ ]:
import threading

counter = 0


def increment():
    global counter
    for _ in range(100000):
        counter += 1


threads = []

for _ in range(2):  # 두 스레드 실행
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("최종 counter 값:", counter)


In [ ]:
import threading

counter = 0
lock = threading.Lock()


def increment():
    global counter
    for _ in range(100000):
        with lock:
            counter += 1


threads = []

for _ in range(2):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("최종 counter 값 (Lock 적용):", counter)


In [ ]:
import threading

counter = 0
semaphore = threading.Semaphore(1)  # binary semaphore (mutex)


def increment():
    global counter
    for _ in range(100000):
        semaphore.acquire()
        counter += 1
        semaphore.release()


threads = []

for _ in range(2):
    t = threading.Thread(target=increment)
    threads.append(t)
    t.start()

for t in threads:
    t.join()

print("최종 counter 값 (Semaphore 적용):", counter)


In [ ]:
import threading
import time

lock = threading.Lock()


def high_priority():
    while True:
        with lock:
            print("[HIGH] Executing critical section")
            time.sleep(0.5)  # 오래 점유
        time.sleep(0.01)  # 거의 쉬지 않음


def low_priority():
    while True:
        got_lock = lock.acquire(timeout=0.1)
        if got_lock:
            try:
                print("[LOW] Finally got the lock!")
            finally:
                lock.release()
            break
        else:
            print("[LOW] Waiting…")


t1 = threading.Thread(target=high_priority, daemon=True)
t2 = threading.Thread(target=low_priority)

t1.start()
t2.start()
t2.join()


In [1]:
import threading
import time
from collections import defaultdict


class StarvationMonitor:
    def __init__(self, threshold_sec=10):
        self.wait_start = defaultdict(float)
        self.lock = threading.Lock()
        self.threshold = threshold_sec

    def start_waiting(self, tid):
        with self.lock:
            self.wait_start[tid] = time.time()

    def stop_waiting(self, tid):
        with self.lock:
            if tid in self.wait_start:
                del self.wait_start[tid]

    def check_starvation(self):
        now = time.time()
        starving = []
        with self.lock:
            for tid, start in self.wait_start.items():
                wait_time = now - start
                if wait_time > self.threshold:
                    starving.append((tid, round(wait_time, 2)))
        return starving


# 테스트
monitor = StarvationMonitor(threshold_sec=5)


def thread_simulation(tid, wait_before_acquire):
    monitor.start_waiting(tid)
    time.sleep(wait_before_acquire)
    monitor.stop_waiting(tid)


# 스레드 3개 시뮬레이션
threading.Thread(target=thread_simulation, args=("T1", 3)).start()
threading.Thread(target=thread_simulation, args=("T2", 7)).start()
threading.Thread(target=thread_simulation, args=("T3", 6)).start()

time.sleep(6)  # 일부 스레드는 아직 대기 중

# starving 스레드 출력
print("Starving Threads:", monitor.check_starvation())


Starving Threads: [('T2', 6.0), ('T3', 6.0)]


In [3]:
import threading


class InventoryManager:
    def __init__(self):
        self.inventory = {"product_123": 1}  # 재고 1개

    def purchase_item(self, product_id: str, customer_id: str):
        # 1. 재고 확인 (READ)
        current_stock = self.inventory.get(product_id, 0)
        print(f"고객 {customer_id}: 현재 재고 {current_stock}")

        if current_stock > 0:
            time.sleep(0.1)  # 네트워크 지연 시뮬레이션
            # 2. 재고 감소 (WRITE) - 여기서 Race Condition 발생!
            self.inventory[product_id] = current_stock - 1
            print(
                f"고객 {customer_id}: 구매 성공! 남은 재고: {self.inventory[product_id]}"
            )
            return True
        else:
            print(f"고객 {customer_id}: 재고 부족")
            return False


# 해결된 코드 (Atomic Operations 사용)


class SafeInventoryManager:
    def __init__(self):
        self.inventory = {"product_123": 1}
        self.lock = threading.Lock()

    def purchase_item(self, product_id: str, customer_id: str):
        with self.lock:  # 원자적 연산 보장
            current_stock = self.inventory.get(product_id, 0)
            print(f"고객 {customer_id}: 현재 재고 {current_stock}")

            if current_stock > 0:
                self.inventory[product_id] = current_stock - 1
                print(
                    f"고객 {customer_id}: 구매 성공! 남은 재고: {self.inventory[product_id]}"
                )
                return True
            else:
                print(f"고객 {customer_id}: 재고 부족")
                return False


In [20]:
a = InventoryManager()

In [29]:
threading.Thread(target=a.purchase_item, args=("product_123", "ab")).start()
threading.Thread(target=a.purchase_item, args=("product_123", "ab")).start()


고객 ab: 현재 재고 0
고객 ab: 재고 부족
고객 ab: 현재 재고 0
고객 ab: 재고 부족


In [30]:
import threading
import time
from collections import deque
from statistics import mean, median

# =========================
# 공통 메트릭 수집 도우미
# =========================


class BenchMetrics:
    """버퍼 내부의 대기/신호 관련 카운터를 수집한다."""

    def __init__(self):
        self.lock = threading.Lock()
        self.waits_on_full = 0  # put 시 버퍼가 가득 차서 기다린 횟수
        self.waits_on_empty = 0  # get 시 버퍼가 비어서 기다린 횟수

    def inc_full(self):
        with self.lock:
            self.waits_on_full += 1

    def inc_empty(self):
        with self.lock:
            self.waits_on_empty += 1

    def snapshot(self):
        with self.lock:
            return dict(
                waits_on_full=self.waits_on_full, waits_on_empty=self.waits_on_empty
            )


# =========================
# 공통 인터페이스
# =========================


class BufferIFace:
    """put/get 인터페이스만 강제. 구현은 세 가지 스타일로 제공."""

    def put(self, item):
        raise NotImplementedError

    def get(self):
        raise NotImplementedError


# =========================
# 1) Mesa Style (표준) 구현
# - 스퓨리어스 웨이크업 & Mesa 의미론 대응: while 재검사
# =========================


class MesaBuffer(BufferIFace):
    def __init__(self, capacity=64, metrics: BenchMetrics | None = None):
        self.cap = capacity
        self.q = deque()
        self.lock = threading.Lock()
        self.cv = threading.Condition(self.lock)
        self.metrics = metrics or BenchMetrics()

    def put(self, item):
        with self.cv:  # CV는 항상 락과 페어
            while len(self.q) >= self.cap:
                self.metrics.inc_full()
                # wait는 원자적으로 unlock→sleep→relock 수행
                self.cv.wait()
            self.q.append(item)
            # 상태 변화(비어있지 않음) 알림: 보통 하나만 깨우는 것이 효율적
            self.cv.notify()

    def get(self):
        with self.cv:
            while not self.q:
                self.metrics.inc_empty()
                self.cv.wait()
            item = self.q.popleft()
            # 상태 변화(가득참→여유) 알림
            self.cv.notify()
            return item


# =========================
# 2) Hoare Style (교육용 시뮬레이션)
# - 실제 파이썬 런타임은 Mesa 시맨틱. urgent 큐를 둬서 "즉시 양도" 느낌을 흉내낸다.
# - 논리: signaler는 urgent 큐로 이동해 대기, waiter가 일을 마치면 urgent.notify()로 깨워 제어권 회수
# =========================


class HoareSimBuffer(BufferIFace):
    def __init__(self, capacity=64, metrics: BenchMetrics | None = None):
        self.cap = capacity
        self.q = deque()
        self.lock = threading.Lock()
        self.waiters = threading.Condition(self.lock)  # 대기자용
        self.urgent = threading.Condition(self.lock)  # 신호자 복귀 대기열
        self.metrics = metrics or BenchMetrics()

    def put(self, item):
        with self.lock:
            while len(self.q) >= self.cap:
                self.metrics.inc_full()
                self.waiters.wait()
            self.q.append(item)
            # 조건 충족 → waiter를 깨우고, "즉시 양도" 시뮬:
            self.waiters.notify()
            # signaler는 urgent 큐에서 잠깐 대기하여 waiter가 먼저 달리게 한다.
            # (임계영역 길이에 따라 오히려 손해일 수 있음 — 교육용!)
            self.urgent.wait(timeout=0)  # 0으로 스핀 없음, 즉시 리턴 (양도 힌트 효과만)

    def get(self):
        with self.lock:
            while not self.q:
                self.metrics.inc_empty()
                self.waiters.wait()
            item = self.q.popleft()
            self.waiters.notify()
            # waiter가 일을 마쳤으니 signaler 깨움(제어권 복원 힌트)
            self.urgent.notify()
            return item


# =========================
# 3) Predicate-based Style
# - wait_for(predicate)로 프레디킷 캡슐화 + 내부 while 재검사 자동화
# =========================


class PredicateBuffer(BufferIFace):
    def __init__(self, capacity=64, metrics: BenchMetrics | None = None):
        self.cap = capacity
        self.q = deque()
        self.lock = threading.Lock()
        self.cv = threading.Condition(self.lock)
        self.metrics = metrics or BenchMetrics()

    def _not_full(self):
        return len(self.q) < self.cap

    def _not_empty(self):
        return len(self.q) > 0

    def put(self, item):
        with self.cv:
            # 실패(타임아웃 없음 가정) 시 False 반환이지만 여기서는 성공할 때까지 대기
            while not self._not_full():
                self.metrics.inc_full()
                self.cv.wait()
            self.q.append(item)
            self.cv.notify()

    def get(self):
        with self.cv:
            while not self._not_empty():
                self.metrics.inc_empty()
                self.cv.wait()
            item = self.q.popleft()
            self.cv.notify()
            return item


# =========================
# 벤치마크 러너
# =========================


def run_benchmark(
    BufferCls,
    *,
    capacity=64,
    num_producers=2,
    num_consumers=2,
    items_per_producer=10,
    producer_think_s=0.0,
    consumer_think_s=0.0,
):
    """
    동일 인자 하에 BufferCls 구현을 벤치마크한다.
    - 각 아이템은 (t0, payload) 형태로 생산되어 소비 시 지연을 측정한다.
    - 소비자마다 목표 소비 개수를 배분하여 자연 종료하도록 한다.
    """
    metrics = BenchMetrics()
    buf = BufferCls(capacity=capacity, metrics=metrics)
    total_items = num_producers * items_per_producer

    # 소비자별 목표 개수 배분(마지막 소비자가 나머지 처리)
    base = total_items // num_consumers
    shares = [base] * num_consumers
    shares[-1] += total_items - base * num_consumers

    start_barrier = threading.Barrier(num_producers + num_consumers)
    latencies = []  # 전체 지연(ms) 수집
    lat_lock = threading.Lock()

    def record_latency_ms(t0):
        with lat_lock:
            latencies.append((time.perf_counter() - t0) * 1000.0)

    def producer(pid):
        start_barrier.wait()
        for i in range(items_per_producer):
            t0 = time.perf_counter()
            buf.put((t0, (pid, i)))  # payload: (producer_id, seq)
            if producer_think_s:
                time.sleep(producer_think_s)

    def consumer(cid, quota):
        start_barrier.wait()
        for _ in range(quota):
            t0, payload = buf.get()
            record_latency_ms(t0)
            if consumer_think_s:
                time.sleep(consumer_think_s)

    producers = [
        threading.Thread(target=producer, args=(p,)) for p in range(num_producers)
    ]
    consumers = [
        threading.Thread(target=consumer, args=(c, shares[c]))
        for c in range(num_consumers)
    ]

    t_begin = time.perf_counter()
    for th in producers + consumers:
        th.start()
    for th in producers + consumers:
        th.join()
    t_end = time.perf_counter()

    elapsed = t_end - t_begin
    m = metrics.snapshot()

    # 지연 통계
    lat_ms = latencies if latencies else [0.0]
    stats = {
        "total_items": total_items,
        "elapsed_s": elapsed,
        "throughput_items_per_s": total_items / elapsed
        if elapsed > 0
        else float("inf"),
        "latency_ms_avg": mean(lat_ms),
        "latency_ms_p50": median(lat_ms),
        "latency_ms_max": max(lat_ms),
        "waits_on_full": m["waits_on_full"],
        "waits_on_empty": m["waits_on_empty"],
    }
    return stats


# =========================
# 실행 예시
# =========================


def pretty(name, stats):
    print(f"\n=== {name} ===")
    print(
        f"items={stats['total_items']}, elapsed={stats['elapsed_s']:.4f}s, "
        f"throughput={stats['throughput_items_per_s']:.1f}/s"
    )
    print(
        f"latency(ms): avg={stats['latency_ms_avg']:.2f}, "
        f"p50={stats['latency_ms_p50']:.2f}, max={stats['latency_ms_max']:.2f}"
    )
    print(
        f"waits: on_full={stats['waits_on_full']}, on_empty={stats['waits_on_empty']}"
    )


if __name__ == "__main__":
    # 벤치마크 파라미터
    CAP = 64
    NP, NC = 4, 4
    ITEMS_PER_PROD = 500
    P_THINK, C_THINK = 0.0, 0.0

    s1 = run_benchmark(
        MesaBuffer,
        capacity=CAP,
        num_producers=NP,
        num_consumers=NC,
        items_per_producer=ITEMS_PER_PROD,
        producer_think_s=P_THINK,
        consumer_think_s=C_THINK,
    )
    pretty("Mesa (표준 while 재검사)", s1)

    s2 = run_benchmark(
        HoareSimBuffer,
        capacity=CAP,
        num_producers=NP,
        num_consumers=NC,
        items_per_producer=ITEMS_PER_PROD,
        producer_think_s=P_THINK,
        consumer_think_s=C_THINK,
    )
    pretty("Hoare (교육용 시뮬레이션)", s2)

    s3 = run_benchmark(
        PredicateBuffer,
        capacity=CAP,
        num_producers=NP,
        num_consumers=NC,
        items_per_producer=ITEMS_PER_PROD,
        producer_think_s=P_THINK,
        consumer_think_s=C_THINK,
    )
    pretty("Predicate(wait_for)", s3)



=== Mesa (표준 while 재검사) ===
items=2000, elapsed=0.0076s, throughput=264309.0/s
latency(ms): avg=0.10, p50=0.08, max=3.82
waits: on_full=82, on_empty=77

=== Hoare (교육용 시뮬레이션) ===
items=2000, elapsed=0.0102s, throughput=196063.1/s
latency(ms): avg=0.18, p50=0.13, max=7.04
waits: on_full=60, on_empty=53

=== Predicate(wait_for) ===
items=2000, elapsed=0.0050s, throughput=402714.3/s
latency(ms): avg=0.07, p50=0.06, max=2.14
waits: on_full=41, on_empty=38
